In [0]:
from pyspark.sql.functions import *

In [0]:
@dlt.table(
    name="bronze.bronze_employees"
)
def bronze_employees():
    return spark.readStream.table("data_engineering_practice.app_data.employees_raw");

In [0]:
@dlt.table (
     name ="silver.silver_employees"
)
@dlt.expect_or_drop("non_null_salary", "salary IS NOT NULL")
@dlt.expect_or_drop("unique_email", "email IS NOT NULL")
def silver_employees():
    df = spark.readStream.table("Live.bronze_employees")
    df = df.dropDuplicates(["email"])
    return df 


In [0]:
@dlt.table(
    name ="gold.gold_employees"
)
def gold_employees():
    df = spark.readStream.table("Live.silver.silver_employees")
    df_summery = df.groupBy("department").agg(
        count("*").alias("employee_count"),
        avg("salary").alias("avg_salary")
    )
    
    return df_summery
